In [1]:
import pandas as pd

In [2]:
test = pd.read_csv('../input/titanic/train.csv')
test.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Let's study the dataset to begin doing the predictions
test.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
test['Ticket'].value_counts()

347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: Ticket, Length: 681, dtype: int64

In [5]:
#those people have the same ticket number, therefore they bought together and probably where traveling together.
group_tickets = test[test['Ticket'].map(test['Ticket'].value_counts() > 1)]
print(group_tickets.shape)

(344, 12)


In [6]:
#There are too many missing values in cabin, they are cabinless people? This was a mistake? Probably, should drop this column.
print('No cabin = %s' %test.Cabin.isna().sum())

No cabin = 687


In [7]:
test.drop(columns=('Cabin'), inplace=True)
test.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [8]:
#Large number os na, but is possible to deal with it.
print('No age = %s' %test.Age.isna().sum())

No age = 177


In [9]:
#I have a choice here, I could try to fill in the nan values in age, either with median, mean or
#imputing values from other catgories to take an avarage mean. Or drop the nans.
#I think the number still will be large enough to train. So I'll go with the drop option.
test.dropna(subset=['Age'], inplace=True)
print('No age = %s' %test.Age.isna().sum())

No age = 0


In [10]:
#Do the place you emkared affects your survival chance? I mean... It shouldn't, but maybe it could affect where
#you will be in the ship.
test['Embarked'].value_counts()

S    554
C    130
Q     28
Name: Embarked, dtype: int64